In [ ]:
import easyocr
import cv2
import os
import numpy as np
from PIL import Image
import pdf2image
import pickle
import torch
import pandas as pd
reader = easyocr.Reader(['en'])
poppler_path = r"D:\Release-23.08.0-0\poppler-23.08.0\Library\bin"

In [ ]:
with open('400_leases_list.pkl', 'rb') as f:
    files_with_400 = pickle.load(f)

df = pd.DataFrame(columns = ['source_sentence','source_file'])

In [ ]:
#main cell to extract the lease information from the lease dir
directory_path = os.getcwd()+"\\leases\\"
for file in os.listdir(directory_path):
    if file.endswith(".pdf") and file in files_with_400:
        path = directory_path+file
        im = pdf2image.convert_from_path(path, poppler_path=poppler_path)

        for i, image in enumerate(im):
            a = np.array(image)
            b = a[100:1000] #page 1, landlord
            c = cv2.cvtColor(b, cv2.COLOR_BGR2GRAY)
            c = cv2.bitwise_not(c)
            results = reader.readtext(c, paragraph = True, batch_size = 4, detail = 0)
            full_text = ' '.join([text for (text) in results])
            # print(full_text)
            if "Form 400" in full_text and "Schedule A" not in full_text:
                sentence = {'source_sentence': full_text, 'source_file': file}
                df = pd.concat([df, pd.DataFrame([sentence])], ignore_index = True, axis = 0)
                print(len(df))

In [14]:
df.to_json(os.getcwd()+"\\data\\source_sentences.json")